In [1]:
# Load the data
import pandas as pd

df = pd.read_csv('Data/reviews.csv', encoding="ISO-8859-1")
df.head()

,Text,Sentiment
0,Once again Mr. Costner has dragged out a movie...,0
1,This is an example of why the majority of acti...,0
2,"First of all I hate those moronic rappers, who...",0
3,Not even the Beatles could write songs everyon...,0
4,Brass pictures (movies is not a fitting word f...,0


In [2]:
# Define a function for cleaning text by removing punctuation characters, converting multiple
# spaces to single spaces, and converting characters to lowercase
import string, re
table = str.maketrans('', '', string.punctuation)

def clean_text(text):
    return re.sub(' +', ' ', text.translate(table).lower())

In [3]:
# Clean the text in the dataset
df['Text'] = df['Text'].apply(lambda x: clean_text(x))
df.head()

,Text,Sentiment
0,once again mr costner has dragged out a movie ...,0
1,this is an example of why the majority of acti...,0
2,first of all i hate those moronic rappers who ...,0
3,not even the beatles could write songs everyon...,0
4,brass pictures movies is not a fitting word fo...,0


In [4]:
# Vectorize the text
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words=['i', 'we', 'you', 'the', 'and', 'am', 'are'], min_df=10)
vectors = vectorizer.fit_transform(df['Text'])

In [5]:
# Split the dataset into a training set and a test set
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(vectors, df['Sentiment'], test_size=0.2, random_state=0)

In [6]:
# Train a classifier
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(x_train, y_train)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [7]:
# Score the model
model.score(x_test, y_test)

0.9085

In [8]:
# Assess accuracy with a confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, model.predict(x_test))

array([[4535,  467],
       [ 448, 4550]])

In [9]:
# Score a review
review = [clean_text('The long lines and poor customer service really turned me off.')]
model.predict_proba(vectorizer.transform(review))[0][1]

0.16786410589781997

In [10]:
# Score another review
review = [clean_text('One of the most delightful experiences I have ever had!')]
model.predict_proba(vectorizer.transform(review))[0][1]

0.838033540534623

In [11]:
# Save the model and the vocabulary
import pickle

pickle.dump(model, open('Data/sentiment_analysis.pkl', 'wb'))
pickle.dump(vectorizer.vocabulary_, open('Data/vocabulary.pkl', 'wb'))